This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]
ResampleType = ttk.ResampleImage[ImageType]

reader1 = ReaderType.New(FileName="data/mra-Brain.mha")
reader1.Update()
im1 = reader1.GetOutput()
res = ResampleType.New(Input = im1) # Vessel extraction requires Isotropic voxel spacing
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

reader2 = ReaderType.New(FileName="data/mri-t1-Brain.mha")
reader2.Update()
im2 = reader2.GetOutput()
res = ResampleType.New(Input = im2)
res.SetMakeHighResIso(True)
res.Update()
im2iso = res.GetOutput()

reader3 = ReaderType.New(FileName="data/mri-t2-Brain.mha")
reader3.Update()
im3 = reader3.GetOutput()
res = ResampleType.New(Input = im3)
res.SetMakeHighResIso(True)
res.Update()
im3iso = res.GetOutput()


In [3]:
# Manually extract a few vessels to form an image-specific training set
xp=[[-3.9,-54.8,12.3],
    [25.1,-20.8,-22.7],
    [-27.0,-25.8,-23.9]]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.2)
vSeg.SetStepX(0.1)
vSeg.SetRadiusInObjectSpace( 1.5 )
vSeg.ExtractTubeInObjectSpace( xp[0], 1 )
vSeg.ExtractTubeInObjectSpace( xp[1], 2 )
vSeg.ExtractTubeInObjectSpace( xp[2], 3 )

tubeMaskImage = vSeg.GetTubeMaskImage()

In [4]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
trMask.SetNotVesselWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [7]:
im1Math = ttk.ImageMath[ImageType,LabelMapType].New(Input=im1iso)
im1Math.Threshold(0,0,63,0)
bkgMask = im1Math.GetOutput()

imCombMath = ttk.ImageMath[LabelMapType,LabelMapType].New(Input=bkgMask)
imCombMath.AddImages(fgMask, 1, 1)
mask = imCombMath.GetOutput()

view(mask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC3; proxy …

In [13]:
segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
segmenter.SetFeatureImage( im1iso )
#segmenter.AddFeatureImage( im2iso )
#segmenter.AddFeatureImage( im3iso )
segmenter.SetInputLabelMap( mask )
segmenter.SetObjectId( 255 )
segmenter.AddObjectId( 127 )
segmenter.AddObjectId( 63 )
segmenter.SetVoidId( 0 )
segmenter.SetErodeDilateRadius( 1 )
segmenter.SetDilateFirst(True)
segmenter.Update()
segmenter.ClassifyImages()

In [14]:
view(segmenter.GetClassProbabilityImage(0))
#imMath = ttk.ImageMath[ImageType,ImageType].New(Input = segmenter.GetClassProbabilityImage(0))
#imMath.AddImages( segmenter.GetClassProbabilityImage(1), 1, -1 )
#view(imMath.GetOutput())

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…